In [169]:
import pandas as pd
import os
from tqdm import tqdm
#from tqdm.notebook import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, image
from keras.applications.densenet import DenseNet169
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import Sequential, Model, load_model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback

In [140]:
# #Utility function to find the list of files in a directory excluding the hidden files.
# def listdir_nohidden(path):
#     for f in os.listdir(path):
#         if not f.startswith('.'):
#             yield f

In [141]:
#     image_data = {}
#     study_label = {'positive': 1, 'negative': 0}
#     category = 'train'
#     study_types = ['XR_ELBOW']
#     #study_types = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
   
#     i = 0
#     image_data[category] = pd.DataFrame(columns=['Path','Label'])
#     for study_type in study_types: # Iterate throught every study types
#         DATA_DIR = 'MURA-v1.1/%s/%s/' % (category, study_type)
#         patients = list(os.walk(DATA_DIR))[0][1]  # list of patient folder names
#         for patient in tqdm(patients):  # for each patient folder
#             for study in os.listdir(DATA_DIR + patient):  # for each study in that patient folder
#                 if(study != '.DS_Store'):
#                     label = study_label[study.split('_')[1]]  # get label 0 or 1
#                     path = DATA_DIR + patient + '/' + study + '/'  # path to this study
#                     for j in range(len(list(listdir_nohidden(path)))):
#                         image_path = path + 'image%s.png' % (j + 1)
#                         image_data[category].loc[i] = [image_path,label]  # add new row
#                         i += 1
#     image_data[category].to_csv(category+"_image_data.csv",index = None, header=False)


In [173]:
def data_loader(category,study_type):
    image_data = {}
    study_label = {'positive': 1, 'negative': 0}
    #study_types = ['XR_ELBOW','XR_FINGER','XR_FOREARM','XR_HAND','XR_HUMERUS','XR_SHOULDER','XR_WRIST']
   
    i = 0
    image_data[category] = pd.DataFrame(columns=['Path','Label'])
    for study_type in study_types: # Iterate throught every study types
        DATA_DIR = 'MURA-v1.1/%s/%s/' % (category, study_type)
        patients = list(os.walk(DATA_DIR))[0][1]
        #print("Patient data - ", patients)# list of patient folder names
        for patient in tqdm(patients):  # for each patient folder
            for study in os.listdir(DATA_DIR + patient):
                #print('Study split',study.split('_')[1])
                label = study_label[study.split('_')[1]]  # get label 0 or 1
                path = DATA_DIR + patient + '/' + study + '/'  # path to this study
                for j in range(len(list(os.listdir(path)))):
                    #print(list(os.listdir(path)))
                    image_path = path + 'image%s.png' % (j + 1)
                    image_data[category].loc[i] = [image_path,label]  # add new row
                    i += 1
    image_data[category].to_csv(category+"_image_data_test.csv",index = None, header=False)

In [144]:
def getImagesInArrayNew(train_dataframe):
    images = []
    labels = []
    for i, data in tqdm(train_dataframe.iterrows()):
        #print('This is number of image',i,data)
        img = cv2.imread(data['Path'])
        img = cv2.resize(img,(img_width,img_height))    
        #plt.imshow(img)
        images.append(img)
        labels.append(data['Label'])
    images = np.asarray(images).astype('float32') 
    #normalization
    mean = np.mean(images[:, :, :])
    std = np.std(images[:, :, :])
    images[:, :, :] = (images[:, :, :] - mean) / std
    labels = np.asarray(labels)
    return {'images': images, 'labels': labels}

In [145]:
# train_image_df = pd.read_csv('train_image_data_test.csv', names=['Path', 'Label'])
# print(train_image_df.head())
# dd={}

# img_width, img_height = 224, 224
# dd['train'] = train_image_df
# #print(dd)
# train_dict = getImagesInArrayNew(train_image_df)

In [174]:
#train_data_dir = 'train/XR_ELBOW'
#valid_data_dir = 'valid/XR_ELBOW'

train_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip=True

)

study_types = ['XR_ELBOW_test']

data_loader('train',study_types)
data_loader('valid',study_types)


train_image_df = pd.read_csv('train_image_data_test.csv', names=['Path','Label'])
valid_image_df = pd.read_csv('valid_image_data_test.csv', names=['Path','Label'])

dd={}

dd['train'] = train_image_df
dd['valid'] = valid_image_df


train_dict = getImagesInArrayNew(train_image_df)
valid_dict = getImagesInArrayNew(valid_image_df)

train_datagen.fit(train_dict['images'],augment=True)
valid_datagen.fit(valid_dict['images'],augment=True)

train_generator = train_datagen.flow(
    x=train_dict['images'],
    y=train_dict['labels']
)

validation_generator = test_datagen.flow(
    x=valid_dict['images'],
    y=valid_dict['labels'],
    batch_size = 1
)

100%|█████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 150.13it/s]
50it [00:00, 151.81it/s]
81it [00:00, 168.78it/s]


In [175]:

#model parameters for training
#K.set_learning_phase(1)
nb_train_samples = len(train_dict['images'])
print('Length of training samples',len(train_dict['images']))
nb_validation_samples = len(valid_dict['images'])
print('Length of validation samples',len(valid_dict['images']))
epochs = 2
batch_size = 8
steps_per_epoch = nb_train_samples//batch_size
print(steps_per_epoch)
n_classes = 1

Length of training samples 50
Length of validation samples 81
6


In [176]:
def build_model():
    base_model = DenseNet169(input_shape=(None, None,3),
                             weights='imagenet',
                             include_top=False,
                             pooling='avg')

    x = base_model.output
    predictions = Dense(n_classes,activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [177]:
model = build_model()

In [178]:
#Compiling the model
model.compile(loss="binary_crossentropy", optimizer='adam', metrics=['acc', 'mse'])

In [179]:
#callbacks for early stopping incase of reduced learning rate, loss unimprovement
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, verbose=1, min_lr=0.0001)
callbacks_list = [early_stop, reduce_lr]

In [180]:
#train the module
model_history = model.fit_generator(
    train_generator,
    epochs=epochs,
    workers=0,
    use_multiprocessing=False,  
    steps_per_epoch = nb_train_samples//batch_size,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples //batch_size,
    #callbacks=callbacks_list
)

Epoch 1/2
6/6 [==============================] - 399s 67s/step - loss: 0.5616 - acc: 0.6894 - mean_squared_error: 0.1942 - val_loss: 1.7324 - val_acc: 0.5000 - val_mean_squared_error: 0.4371
Epoch 2/2
6/6 [==============================] - 687s 115s/step - loss: 0.4748 - acc: 0.8608 - mean_squared_error: 0.1208 - val_loss: 2.9261 - val_acc: 0.4000 - val_mean_squared_error: 0.5262
